In [ ]:
import torch
from torch import nn
from torchvision.transforms import v2
from torch.utils.data import DataLoader, random_split
from torchmetrics import Accuracy, F1Score


import lightning as L
from lightning.pytorch.loggers import CSVLogger
from lightning import Trainer


import numpy as np
import matplotlib.pyplot as plt

from Datasets.Simpson_dataset import SimpsonDataset


from Models.VGG.VGG16 import VGG

In [2]:
class CNN(L.LightningModule):
    def __init__(self, model: nn.Module, criterion: nn.Module):
        super().__init__()
        self.model = model
        self.criterion = criterion
        self.accuracy_score = Accuracy(task="multiclass", num_classes=42)
        self.f1_score = F1Score(task="multiclass", num_classes=42)

    def training_step(self, batch, batch_idx):
        data, target = batch
        out = self(data)
        loss = self.criterion(out, target)
        self.log('train_loss', loss, on_step=False, on_epoch=True, prog_bar=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        data, target = batch
        out = self(data)
        val_loss = self.criterion(out, target)

        acc = self.accuracy_score(out, target)
        f1 = self.f1_score(out, target)
        self.log('val_loss', val_loss, prog_bar=True, on_step=False, on_epoch=True)
        self.log('val_acc', acc, prog_bar=True, on_step=False, on_epoch=True)
        self.log('val_f1', f1, prog_bar=True, on_step=False, on_epoch=True)
    
    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters())
        return optimizer
    
    def forward(self, data):
        return self.model(data)

In [3]:
device = torch.device('cuda:0')

train_transforms = v2.Compose([
    v2.RandomResizedCrop(size=(224, 224), antialias=True),
    v2.RandomHorizontalFlip(p=0.5),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

val_transforms = v2.Compose([
    v2.RandomResizedCrop(size=(224, 224), antialias=True),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

full_transforms = v2.Compose([v2.ToDtype(torch.float32, scale=True)])


full_dataset = SimpsonDataset(
    path_to_dataset=r'D:\ethd\ml\Classic-cnn-archs\data',
    mode='TRAIN',
    transforms=train_transforms,
    device=device)

train_size = int(0.8 * len(full_dataset))
valid_size = len(full_dataset) - train_size
train_dataset, valid_dataset = random_split(full_dataset, [train_size, valid_size])

valid_dataset.dataset.transform = val_transforms

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=5)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False, num_workers=5)

In [ ]:
epoch_num = 10
cnn = VGG(num_classes=42)
criterion = torch.nn.CrossEntropyLoss()
model = CNN(cnn, criterion=criterion)

csv_logger = CSVLogger(save_dir='logs/', name="VGG16")

trainer = Trainer(
    logger=csv_logger,
    max_epochs=epoch_num,
    accelerator = 'cuda'
)

trainer.fit(model, train_loader, valid_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type               | Params | Mode 
--------------------------------------------------------------
0 | model          | AlexNet            | 58.1 M | train
1 | criterion      | CrossEntropyLoss   | 0      | train
2 | accuracy_score | MulticlassAccuracy | 0      | train
3 | f1_score       | MulticlassF1Score  | 0      | train
--------------------------------------------------------------
58.1 M    Trainable params
0         Non-trainable params
58.1 M    Total params
232.252   Total estimated model params size (MB)
37        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

d:\ethd\conda\envs\laba-env\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:420: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.


d:\ethd\conda\envs\laba-env\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:420: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.


Epoch 0:  13%|█▎        | 70/524 [00:11<01:15,  5.98it/s, v_num=3]


Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

: 